In [ ]:
import os
import re
import warnings
import numpy as np
import pandas as pd
import matplotlib.cbook
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.collections import PolyCollection
warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

In [ ]:
prefix = 'high_08yr'
TSUN = 4.9169e-6
gbfisher_columns = ['f','colatitude','longitude','amplitude','inclination','polarization','phase','fdot','fddot','sigma_df/f','sigma_colatitude','sigma_longitude','sigma_dA/A','sigma_inclination','sigma_polarization','sigma_phase','sigma_dfdot/fdot','sigma_dfddot/fddot','sigma_Omega','SNR','b','l','mchirp','distance'] 
gbfisher = pd.read_table(f'../../ldasoft/gbfisher_example/{prefix}_gbfisher_parameters.dat',delimiter=' ', header=None,index_col=False,names=gbfisher_columns)
gbfisher['inclination'] = 90 - abs(np.degrees(gbfisher['inclination']) - 90)
gbfisher['sigma_mchirp'] = (1/(5*TSUN))*((5/3)**(3/5))*(np.pi**(-8/5))*(gbfisher['f']**(-11/5))*(gbfisher['fdot']**(3/5))*((121*(gbfisher['sigma_df/f']**2) + 9*(gbfisher['sigma_dfdot/fdot']**2))**(1/2))

In [ ]:
def readData(raw, index, inFunc=None):
    x = []
    for pos, line in enumerate(raw):
        values = [float(x) for x in line.split()]
        if inFunc==None:
            x.append(values[index])
        else:
            x.append(inFunc(values[index]))
    return x

def plot_quantiles(vplt, vqt, htv, hwt, color):
    bqts = []
    for b in range(len(cut)):
        qts = []
        for c in range(len(vqt)):
            path = PolyCollection.properties(vplt['bodies'][b])['paths'][0].vertices
            cpath = path[[i > wt / 100 for i in (np.abs(path.T[0]-ctv[b]))]].T
            vmin = np.array([np.abs(v - vqt[c][b]) for v in cpath[1]])
            ind = np.where(vmin == np.min(vmin))[0]
            hmax = np.array([np.abs(h - htv[b]) for h in cpath[0][ind]])
            qts.append(cpath[0][ind[np.where(hmax == np.max(hmax))[0]]][0])
        bqts.append(qts)
    bqts = np.array(bqts).T
    plt.hlines(vqt[1],bqts[1],ctv,linewidth=2.0,color=color)
    plt.hlines(vqt[0],bqts[0],ctv,linewidth=1.5,linestyle='--',color=color)
    plt.hlines(vqt[2],bqts[2],ctv,linewidth=1.5,linestyle='--',color=color)

In [ ]:
prior_types = ['em','gw']
exclude = ['fdot','inc']
for c,p in enumerate(prior_types):
    binDir = os.path.join(os.getcwd(),f'../{prefix}-{p}prior')
    for binary in os.listdir(binDir):
        rawData = []
        for folder in os.listdir(os.path.join(binDir,binary)):
            if os.path.isdir(os.path.join(binDir,binary,folder)) & (not folder in exclude):
                if int(re.search('row(.+?)_incl',folder).group(1)) == 0:
                    B_n = int(re.search('binary(.+?)row',folder).group(1))
                    with open(os.path.join(binDir,binary,folder,'post_equal_weights.dat'),'r') as raw:
                        rawData = np.concatenate([rawData,readData(raw,4,inFunc=None)])
        if (B_n == 1 and c==0):
            vdata = pd.DataFrame(np.sort(rawData))
            vdata.insert(0, 'B_n', B_n)
            vdata.insert(0, 'type', f'{p}')
        else:
            raw_vdata = pd.DataFrame(np.sort(rawData))
            raw_vdata.insert(0, 'B_n', B_n)
            raw_vdata.insert(0, 'type', f'{p}')
            vdata = pd.concat([vdata,raw_vdata], axis=0)

In [ ]:
chainDir = f'../../data/{prefix}_results/'
for binary in os.listdir(chainDir):
    rawData = []
    B_n = int(re.search('binary(.*)',binary).group(1))
    with open(os.path.join(chainDir,binary,'chains/dimension_chain.dat.1'),'r') as raw:
        rawData = np.concatenate([rawData,readData(raw,5,inFunc=lambda x: 90-abs(np.degrees(np.arccos(x))-90))])
    raw_vdata = pd.DataFrame(np.sort(rawData))
    raw_vdata.insert(0, 'B_n', B_n)
    raw_vdata.insert(0, 'type', 'gbf')
    vdata = pd.concat([vdata,raw_vdata], axis=0)

In [ ]:
trvs = list(gbfisher['inclination'])
vdata = vdata.rename(columns={0:'value'})
vdata.insert(3, 'true_value', 0)
vdata['true_value'] = np.asarray(trvs)[vdata['B_n']-1]
vdata = vdata.sort_values(['true_value','type','value'])
vdata['value'] = vdata['value'] - vdata['true_value']
vdata['B_n'] = vdata['B_n'].map(dict(zip(vdata['B_n'].unique(),np.arange(1,len(vdata['B_n'].unique())+1))))
b_raw = np.array([np.unique(c).tolist() for c in [vdata.loc[(vdata['B_n'] == b), 'B_n'] for b in vdata['B_n'].unique()]])
b_n = np.unique(b_raw[b_raw.astype(bool)].tolist()).tolist()
gbf = [vdata.loc[(vdata['B_n'] == b) & (vdata['type'] == 'gbf'),'value'] for b in b_n]
em = [vdata.loc[(vdata['B_n'] == b) & (vdata['type'] == 'em'), 'value'] for b in b_n]
gw = [vdata.loc[(vdata['B_n'] == b) & (vdata['type'] == 'gw'), 'value'] for b in b_n]
tv = vdata['true_value'].unique()

In [ ]:
cut = [0,1,2,3,4,5,6,7,8,9]
qgbf = [[0.0001,0.9999] for i in range(len(cut))]
qem = [[0.0001,0.9999] for i in range(len(cut))]
qgw = [[0.0001,0.9999] for i in range(len(cut))]
wt = 4
#######################################################
fig = plt.figure(figsize=(36,18))
ax = fig.add_subplot(111)
ctv = tv[cut]

sgbf = [pd.Series(l) for l in (np.asarray([s.tolist() for s in gbf])[cut])]
sem = [pd.Series(l) for l in (np.asarray([s.tolist() for s in em])[cut])]
sgw = [pd.Series(l) for l in (np.asarray([s.tolist() for s in gw])[cut])]
cgbf = [list(l) for l in [s[s.between(s.quantile(qgbf[i][0]),s.quantile(qgbf[i][1]))] for i,s in enumerate(sgbf)]]
cem = [list(l) for l in [s[s.between(s.quantile(qem[i][0]),s.quantile(qem[i][1]))] for i,s in enumerate(sem)]]
cgw = [list(l) for l in [s[s.between(s.quantile(qgw[i][0]),s.quantile(qgw[i][1]))] for i,s in enumerate(sgw)]]
mgbf = np.array([list(np.percentile(list(m),[10,50,90])) for m in sgbf]).T
mem = np.array([list(np.percentile(list(m),[10,50,90])) for m in sem]).T
mgw = np.array([list(np.percentile(list(m),[10,50,90])) for m in sgw]).T

# Plot GW prior
vpgbf = plt.violinplot(cgbf, positions=ctv, widths=wt, showextrema=False)
for pc in vpgbf['bodies']:
    pc.set_facecolor('#20BB00')
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0],-np.inf,np.mean(pc.get_paths()[0].vertices[:, 0]))
plot_quantiles(vpgbf, mgbf, ctv, wt, '#105E00')

# Plot EM posterior
vpem = plt.violinplot(cem, positions=ctv, widths=wt, showextrema=False)
for pc in vpem['bodies']:
    pc.set_facecolor('#E67E00')
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0],-np.inf,np.mean(pc.get_paths()[0].vertices[:, 0]))
plot_quantiles(vpem, mem, ctv, wt, '#733F00')

# Plot GW+EM posterior
vpgw = plt.violinplot(cgw, positions=ctv, widths=wt, showextrema=False)
for pc in vpgw['bodies']:
    pc.set_facecolor('#226EF1')
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0],np.mean(pc.get_paths()[0].vertices[:, 0]),np.inf)
plot_quantiles(vpgw, mgw, ctv, wt, '#113779')

plt.legend([vpgbf['bodies'][0],vpem['bodies'][0],vpgw['bodies'][0]],['GW Prior','EM Posterior','GW+EM Posterior'],loc='upper right',prop={'size': 24})
plt.title(f'{prefix} Residual Inclination vs True Inclination',fontsize=40,pad=20)
plt.xlabel('True Inclination (degrees)',fontsize=30,labelpad=10)
plt.ylabel('Residual Inclination (degrees)',fontsize=30,labelpad=10)

ax.set_xlim(np.max([0,np.floor(ax.get_xlim()[0])]),np.min([90,np.ceil(ax.get_xlim()[1])]))
for s in range(3,6): 
    scale = 3
    if ((ax.get_xlim()[1] - ax.get_xlim()[0]) % s == 0): 
        scale = s + 1

xticks = np.linspace((ax.get_xlim()[0]),(ax.get_xlim()[1]), scale)
xtls = np.concatenate(([f'{x:.0f}' for x in xticks],[f'{x:.3f}' for x in ctv]))
plt.xticks(np.sort(np.append(xticks,ctv)), fontsize=20)
ax.set_xticklabels([x for _, x in sorted(zip(np.append(xticks,ctv),xtls))])
plt.yticks(fontsize=20)
for v in ctv:
    ax.axvline(v, c='k', linestyle=':', linewidth=2.0)
ax.axhline(0, c='#555555', linestyle = '-', linewidth=1.5)
plt.grid()
plt.show()